# Libraries importing

In [1]:
import numpy as np
import os
import pickle
import tensorflow as tf
from tensorflow_model_optimization.quantization.keras import vitis_inspect, vitis_quantize
from tensorflow_model_optimization.python.core.quantization.keras.vitis.utils import model_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, SGD

2025-01-08 04:27:38.880377: I tensorflow/core/platform/cpu_feature_guard.cc:186] This TensorFlow binary is optimized with Zen Deep Neural Network Library (ZenDNN) to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Function to load the CIFAR10 dataset (after downloading it locally)

In [2]:
def load_cifar10(data_dir):
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict

    # Load training data
    x_train, y_train = [], []
    for i in range(1, 6):  # CIFAR-10 training data consists of 5 batches
        batch = unpickle(os.path.join(data_dir, f"data_batch_{i}"))
        x_train.append(batch[b'data'])
        y_train.extend(batch[b'labels'])

    x_train = np.vstack(x_train).reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    y_train = np.array(y_train)

    # Load test data
    test_batch = unpickle(os.path.join(data_dir, "test_batch"))
    x_test = test_batch[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    y_test = np.array(test_batch[b'labels'])

    return (x_train, y_train), (x_test, y_test)

# Actually loading CIFAR10

In [3]:
#CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = load_cifar10("cifar-10-batches-py")
print(train_images.shape)
print(train_labels.shape)
train_images = train_images.reshape(train_images.shape[0], 32, 32, 3) #ensure shape 32 W x 32 H x 3 channels for each image
test_images = test_images.reshape(test_images.shape[0], 32, 32, 3)

#range 0-1
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

#One-hot encoding labels
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

print(train_images.shape)
print(train_labels.shape)

(50000, 32, 32, 3)
(50000,)
(50000, 32, 32, 3)
(50000, 10)


# Loading a trained model

In [14]:
input_shape = (None,32, 32, 3)
model = load_model('best_model.h5')
model.build(input_shape=input_shape)
model.summary()

opt = SGD(learning_rate=0.1, momentum=0.9, weight_decay=1e-4)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   1792        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 re_lu (ReLU)                   (None, 32, 32, 64)   0           ['batch_normalization[0][0]']

 re_lu_8 (ReLU)                 (None, 16, 16, 128)  0           ['add_3[0][0]']                  
                                                                                                  
 conv2d_11 (Conv2D)             (None, 8, 8, 256)    295168      ['re_lu_8[0][0]']                
                                                                                                  
 batch_normalization_11 (BatchN  (None, 8, 8, 256)   1024        ['conv2d_11[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 re_lu_9 (ReLU)                 (None, 8, 8, 256)    0           ['batch_normalization_11[0][0]'] 
                                                                                                  
 conv2d_12 (Conv2D)             (None, 8, 8, 256)    590080      ['re_lu_9[0][0]']                
          

 dense (Dense)                  (None, 10)           5130        ['global_average_pooling2d[0][0]'
                                                                 ]                                
                                                                                                  
Total params: 11,188,362
Trainable params: 11,178,762
Non-trainable params: 9,600
__________________________________________________________________________________________________


# Copying the guide, 17:57 in the video (just inspection)

In [15]:
target_dpu = 'DPUCZDX8G_ISA1_B4096'
inspector = vitis_inspect.VitisInspector(target = target_dpu)

if not os.path.exists("inspect"):
    os.makedirs("inspect")

inspector.inspect_model(model,
                        input_shape = input_shape,
                        dump_model = True,
                        dump_model_file = "inspect/inspected_model.h5",
                        plot = True,
                        plot_file = "inspect/model_plot.svg",
                        dump_results = True,
                        dump_results_file = "inspect/inspect_results.txt",
                        verbose = 0)

[VAI INFO] Update include_bias_corr: False
[VAI INFO] Update include_fast_ft: False
[VAI INFO] Update include_cle: False
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/root/tensorflow_model_optimization/tmp_sub_graph_quantized_20250108-051022-562493_sCqTBW.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/root/tensorflow_model_optimization/tmp_sub_graph_xnnc_20250108-051022-562493_sCqTBW.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/root/tensorflow_model_optimization/tmp_sub_graph_quantized_20250108-051022-562493_sCqTBW.h5
[INFO] keras version: 2.12.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 6/6 [00:00<00:00, 7931.24it/s]              
I20250108 05:10:22.702638    99 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20250108 05:10:22.702656    99 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20250108 05:10:22.702661    99 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B4096
I20250108 05:10:22.702755    99 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: model_31, with op num: 6
I20250108 05:10:22.702759    99 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20250108 05:10:22.704691    99 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20250108 05:10:22.704711    99 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/root/tensorflow_model_optimization/tmp_sub_graph_xnnc_20250108-051022-562493_sCqTBW.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/root/tensorflow_model_optimization/tmp_sub_graph_quantized_20250108-051022-562862_FWgicf.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/root/tensorflow_model_optimization/tmp_sub_graph_xnnc_20250108-051022-562862_FWgicf.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/root/tensorflow_model_optimization/tmp_sub_graph_quantized_20250108-051022-562862_FWgicf.h5
[INFO] keras version: 2.12.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 5/5 [00:00<00:00, 3572.05it/s]              
I20250108 05:10:22.821817    99 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20250108 05:10:22.821832    99 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20250108 05:10:22.821835    99 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B4096
I20250108 05:10:22.821924    99 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: model_32, with op num: 9
I20250108 05:10:22.821928    99 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20250108 05:10:22.824172    99 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20250108 05:10:22.824189    99 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/root/tensorflow_model_optimization/tmp_sub_graph_xnnc_20250108-051022-562862_FWgicf.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/root/tensorflow_model_optimization/tmp_sub_graph_quantized_20250108-051022-562862_FWgicf.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/root/tensorflow_model_optimization/tmp_sub_graph_xnnc_20250108-051022-562862_FWgicf.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/root/tensorflow_model_optimization/tmp_sub_graph_quantized_20250108-051022-562862_FWgicf.h5
[INFO] keras version: 2.12.0
[INFO] Tensorflow Keras model type: functional


...

[INFO] dump xmodel: /tmp/root/tensorflow_model_optimization/tmp_sub_graph_xnnc_20250108-051022-563069_eTyjrs.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/root/tensorflow_model_optimization/tmp_sub_graph_quantized_20250108-051022-563069_eTyjrs.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/root/tensorflow_model_optimization/tmp_sub_graph_xnnc_20250108-051022-563069_eTyjrs.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/root/tensorflow_model_optimization/tmp_sub_graph_quantized_20250108-051022-563069_eTyjrs.h5
[INFO] keras version: 2.12.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|█| 7/7 [00:00<00:00, 11197.61it/s]             
I20250108 05:10:25.314898    99 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20250108 05:10:25.314910    99 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20250108 05:10:25.314913    99 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B4096
I20250108 05:10:25.314970    99 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: model_61, with op num: 7
I20250108 05:10:25.314973    99 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20250108 05:10:25.316779    99 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 4, DPU subgraph number 1
I20250108 05:10:25.316797    99 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/root/tensorflow_model_optimization/tmp_sub_graph_xnnc_20250108-051022-563069_eTyjrs.xmodel
[VAI INFO] Inspect Results:
[MODEL INFO]:
________________________________________________________________________________________________________________________
Model Name: model
________________________________________________________________________________________________________________________
ID          Name                    Type                    Device      Notes                                           
0/67        input_41                InputLayer              INPUT                                                       
------------------------------------------------------------------------------------------------------------------------
1/67        conv2d                  Conv2D<relu>            DPU                                                         
--------------------------------------------------------------------------------------------------

[VAI INFO] Inspected model has been dumped in: inspect/inspected_model.h5.
[VAI INFO] Start dumping inspected results to inspect/inspect_results.txt
[VAI INFO] Inspected results has been dumped in: inspect/inspect_results.txt.
[VAI INFO] Inspect Finished.


{<keras.engine.input_layer.InputLayer at 0x738e9f4ff040>: <tensorflow_model_optimization.python.core.quantization.keras.vitis.vitis_inspect.InspectResult at 0x738f19173f10>,
 <keras.layers.convolutional.conv2d.Conv2D at 0x738e9f58f970>: <tensorflow_model_optimization.python.core.quantization.keras.vitis.vitis_inspect.InspectResult at 0x738f191738b0>,
 <keras.layers.normalization.batch_normalization.BatchNormalization at 0x738e9f58f520>: <tensorflow_model_optimization.python.core.quantization.keras.vitis.vitis_inspect.InspectResult at 0x738f19173760>,
 <keras.layers.activation.relu.ReLU at 0x738e9f583e80>: <tensorflow_model_optimization.python.core.quantization.keras.vitis.vitis_inspect.InspectResult at 0x738f19111b80>,
 <keras.layers.convolutional.conv2d.Conv2D at 0x738f1938eb20>: <tensorflow_model_optimization.python.core.quantization.keras.vitis.vitis_inspect.InspectResult at 0x738f19111400>,
 <keras.layers.normalization.batch_normalization.BatchNormalization at 0x738f1938eac0>: <ten

# Quantize model

In [16]:
target_dpu = 'DPUCZDX8G_ISA1_B4096'
# target_dpu = "default_optimized.json"
calib_data = train_images[:100]
# Create a tf.data.Dataset and ensure batch dimension
dataset = tf.data.Dataset.from_tensor_slices(calib_data) # convert 100,32,32,3 to a 100 item iterable/list
""" for item in dataset:
    print(item) """
# no need for data calibration because train_images was already calibrated earlier
dataset = dataset.batch(1) # group 5 images each time to check activation and weight distribution

# Inspect calibration data
""" for sample in dataset.take(100):
    print("Calibration sample shape:", sample.shape)
    print("Sample data range:", sample.numpy().min(), sample.numpy().max()) """

# Quantization
try:
    quantizer = vitis_quantize.VitisQuantizer(model, target=target_dpu) #int8 accordiing to docs
    q_model = quantizer.quantize_model(calib_dataset=dataset, 
                                       input_shape=input_shape, 
                                       calib_steps=None) #use all of the dataset
    q_model.save("quantized_model.h5")
    print("Quantized model saved successfully.")
except Exception as e:
    print("Error during quantization:", e)

[VAI INFO] Using func format quantizer
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 3s 345ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...


2025-01-08 05:10:35.997230: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [100,32,32,3]
	 [[{{node Placeholder/_0}}]]


100/100 [==============================] - 17s 104ms/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quant Model Refinement...
[VAI INFO] Start Quantize Position Ajustment...
[VAI INFO] Quantize Position Ajustment Done.
1/1 [==============================] - 1s 1s/step
[VAI INFO] Post-Quant Model Refninement Done.
[VAI INFO] Start Model Finalization...
[VAI INFO] Model Finalization Done.
[VAI INFO] Quantization Finished.
Quantized model saved successfully.


# Recompile & evaluate quantized model

In [17]:
batch_size = 1
#i dont think we need this - Zé 

#learning_rate = 0.0001
#momentum = 0
#epsilon = 1e-08

""" q_model.compile(
    optimizer = tf.keras.optimizers.RMSprop(
        learning_rate = learning_rate,
        momentum = momentum,
        epsilon = epsilon),
    loss = tf.keras.losses.CategoricalCrossentropy(reduction = tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE),
    metrics = ['acc'])
 """

# Instead:
#compile with a dummy optimizer, since we are not training 
q_model.compile(optimizer='sgd',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

q_model.evaluate(test_images, test_labels, batch_size=batch_size)

10000/10000 [==============================] - 181s 18ms/step - loss: 0.5209 - accuracy: 0.9143


[0.5209415555000305, 0.9143000245094299]

# Compile the DPU

In [18]:
!vai_c_tensorflow2 -m "./quantized_model.h5" -a "arch.json" -o "xmodel" -n final_model 

**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized_model.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/final_model_0x101000007010407_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/amd-kria-running-CIFAR-10-vitis_optimization/quantized_model.h5
[INFO] keras version: 2.12.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 87/87 [00:00<00:00, 1019.70it/s]            
[INFO] dump xmodel: /tmp/final_model_0x101000007010407_org.xmodel
[UNILOG][INFO] Compile mode: dpu
[UNILOG][INFO] Debug mode: null
[UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B4096_0101000007010407
[UNILOG][INFO] Graph name: model, with op num: 173
[UNILOG][INFO] Begin to compile...
[UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
[UNILOG][INFO] C